# Hipótese 1 
Neste Jupyter Notebook, exploraremos a hipótese de que, durante os meses de inverno (junho a setembro), as baixas temperaturas e o enfraquecimento do sistema imunológico contribuem para um aumento na utilização do plano de saúde por colaboradores que buscam consultas e tratamentos médicos nas áreas pneumológicas.

***
## Instalação das bibliotecas:
Instala bibliotecas necessárias:

In [ ]:
!pip install pandas numpy

***
## Importação das bibliotecas:
Importa bibliotecas necessárias:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

***
## Importe do banco de dados via arquivo local
Importação do DataFrame da base de dados da Unipar com a biblioteca pandas.<br> 
Para rodar o notebook corretamente, será necessário importar o arquivo de banco de dados manualmente, pois ele não está incluído no repositório devido à presença de dados sensíveis.

#### Passos para Importar o Banco de Dados no VS Code

4. *Obtenha o Arquivo de Dados*:
   - O arquivo BASE_DE_SINISTRO_UNIPAR_BRADESCO_ORIGINAL.csv deverá ser fornecido separadamente. Entre em contato com o responsável pelo projeto para receber o arquivo.

5. *Posicione o Arquivo na Pasta Correta*:
   - Após receber o arquivo, arraste-o para a mesma pasta onde o notebook está localizado em seu computador. Isso garante que o caminho relativo na função de leitura permaneça o mesmo e funcione corretamente.
   
6. *Verifique o Caminho do Arquivo*:
   - O código de leitura do arquivo já está implementado no notebook e não precisa ser alterado:
     python
     df = pd.read_csv('BASE_DE_SINISTRO_UNIPAR_BRADESCO_ORIGINAL.csv', decimal=',')
     
   - Certifique-se de que o arquivo CSV esteja no mesmo diretório que o notebook para evitar problemas de caminho.

7. *Rodar o Notebook*:
   - Com o arquivo posicionado corretamente, execute o notebook normalmente. O pandas irá carregar os dados e você poderá seguir com a análise.

*Nota*: Caso o arquivo não esteja na mesma pasta que o notebook, o código não será capaz de localizar o banco de dados, resultando em um erro. Portanto, é essencial que o arquivo seja arrastado para o diretório correto antes da execução.

Agora que você baixou os notebooks e o banco de dados, pode seguir para os próximos passos de execução, seja localmente ou no Google Colab.

In [3]:
df = pd.read_csv('[NÃO EDITAR] BASE DE SINISTRO UNIPAR_BRADESCO_ORIGINAL.xlsx - BASE.csv', decimal=',')

***
## Pré-Processamento
Como apresentado no notebook 'Pré-Processamento', as linhas a seguir executam várias etapas para limpar e organizar o banco de dados.<br>
Os comentários no código explicam cada ação.

In [4]:
# Tratamento de valores nulos
df = df.dropna()
# Correção do ponto faltante em 'UNIPAR INDUPA DO BRASIL S.A'
df = df.replace({"UNIPAR INDUPA DO BRASIL S.A": "UNIPAR INDUPA DO BRASIL S.A."})
# Remoção de AGREGADO e DEPENDENTE
df_remove_d = df.loc[(df['Elegibilidade Sinistro'] == 'DEPENDENTE') ]
df = df.drop(df_remove_d.index)
df_remove_a = df.loc[(df['Elegibilidade Sinistro'] == 'AGREGADO') ]
df = df.drop(df_remove_a.index)
# Tratamento de valores duplicados
df = df.drop_duplicates(keep='last')
# Convertendo a coluna 'Dt Data Sinistro' para o formato 'YYYYMMDD' (Ano-Mês-Dia)
df['Dt Data Sinistro'] = pd.to_datetime(df['Dt Data Sinistro'], format='%d/%m/%Y').dt.strftime('%Y%m%d')

In [5]:
# Converter a coluna para datetime
df['Dt Data Sinistro'] = pd.to_datetime(df['Dt Data Sinistro'], format='%Y%m%d')

# Extrair componentes de ano, mês e dia
df['Ano'] = df['Dt Data Sinistro'].dt.year
df['Mes'] = df['Dt Data Sinistro'].dt.month
df['Dia'] = df['Dt Data Sinistro'].dt.day

***
## Comprovação da hipótese

A seguir, será apresentado um gráfico que mostra a distribuição das ativações de sinistros por mês, considerando os anos de 2023 e 2024. Essa análise será baseada na coluna 'Descrição Serviço Sinistro'.

In [ ]:
df['Mes_Ano'] = df['Mes'].astype(str) + '-' + df['Ano'].astype(str)
sinistros_por_mes = df.groupby('Mes')['Descricao Servico Sinistro'].count()

sinistros_por_mes = sinistros_por_mes.reset_index()

plt.figure(figsize=(10, 6))
plt.plot(sinistros_por_mes['Mes'], sinistros_por_mes['Descricao Servico Sinistro'], marker='o', linestyle='-', color='g')

plt.xticks(np.arange(1, 13, 1), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

plt.title('Número de ativações de sinistro por mês')
plt.xlabel('Mês')
plt.ylabel('Número de ativações')
plt.grid(True)

plt.show()

Esse gráfico revela um aumento significativo no uso do plano de saúde durante os meses de julho e agosto, o que por si só já nos fornece informações relevantes. A seguir, vamos analisar a distribuição mensal de algumas palavras-chave relacionadas a doenças respiratórias dentro dos nossos dados. Para essa análise, criamos uma variável contendo 44 palavras que podem estar presentes na coluna 'Descrição Serviço Sinistro', e utilizaremos essas palavras para aprofundar nossa análise.

In [7]:
keywords = [
    'resp', 'asma', 'bronq', 'pneumo', 'tosse', 'alerg', 'sinus', 'pulm', 'influ', 'grip', 
    'rinite', 'faring', 'laring', 'otite', 'bronquiolite', 'enfisema', 'COVID', 'corona', 
    'infecc', 'hipox', 'SARS', 'tuberculose', 'mucos', 'ventilação', 'apneia', 'pneumotórax',
    'bronquiectasia', 'hemoptise', 'tracheite', 'epiglotite', 'pleur', 'broncoespasmo', 
    'hiperreactividade brônquica', 'tosse crônica', 'edema pulmonar', 'DPOC', 'fibrose pulmonar',
    'aspiração', 'traqueostomia', 'ventilação não invasiva', 'oxigenoterapia', 'pneumonectomia',
    'laringotraqueobronquite', 'aspirado traqueal'
]

***
## Ativações fora dos meses de inverno

Aqui aplicamos um filtro para que o "df_total_doencas" mantenha apenas as linhas em que a coluna "Descrição Servico Sinistro" contém alguma das palavras presentes na variável "keywords".

In [8]:
df_total_doencas = df[df['Descricao Servico Sinistro'].str.contains('|'.join(keywords), case=False, na=False)]

Número total de linhas:

In [ ]:
print(f'Total de linhas: {df_total_doencas['Descricao Servico Sinistro'].shape[0]}')

Em seguida é plotado um gráfico que agruda os valores nos meses em que eles ocorreram.

In [ ]:
doencas_por_mes = df_total_doencas.groupby('Mes')['Descricao Servico Sinistro'].count().reset_index()

plt.figure(figsize=(10, 6))
plt.scatter(doencas_por_mes['Mes'], doencas_por_mes['Descricao Servico Sinistro'], color='b')

plt.plot(doencas_por_mes['Mes'], doencas_por_mes['Descricao Servico Sinistro'], linestyle='-', color='b')

plt.title('Dispersão das ativações relacionadas a doenças respiratórias por mês')
plt.xlabel('Mês')
plt.ylabel('Número de ativações')
plt.xticks(np.arange(1, 13, 1), ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
plt.grid(True)
plt.savefig('ativacoes_doencas_respiratorias_por_mes.png', dpi=300, bbox_inches='tight')
plt.show()

Como podemos observar, similar ao gráfico que mostra a distribuição das ativações de sinistros por mês, este gráfico também destaca uma grande incidência de sinistros com descrições que contêm as palavras-chave mencionadas anteriormente, especialmente no mês de agosto.

É importante ressaltar que, após o filtro aplicado, o banco de dados contém um total de 321 registros.

***
## Ativações durante os meses de inverno

Na etapa final da comprovação dessa hipótese, filtramos novamente nossos dados para garantir que o "df_inverno" contenha apenas os registros dos meses de junho a setembro, que correspondem ao período do inverno.

In [ ]:
df_inverno = df.loc[((df['Ano'] == 2023) | (df['Ano'] == 2024)) & (df['Mes'] >= 6) & (df['Mes'] <= 9)]
df_inverno.shape[0]

Filtramos novamente utilizando as palavras-chaves

In [ ]:
df_doencas_respiratorias = df_inverno[df_inverno['Descricao Servico Sinistro'].str.contains('|'.join(keywords), case=False, na=False)]
df_doencas_respiratorias['Descricao Servico Sinistro']

Número total de linhas

In [ ]:
print(f'Total de linhas: {df_doencas_respiratorias['Descricao Servico Sinistro'].shape[0]}')

E por fim plotamos os gráficos com as ativações e seus respectivos meses.

In [ ]:
sinistros_inverno_por_mes = df_doencas_respiratorias.groupby('Mes')['Descricao Servico Sinistro'].count().reset_index()
plt.figure(figsize=(10, 6))
plt.plot(sinistros_inverno_por_mes['Mes'], sinistros_inverno_por_mes['Descricao Servico Sinistro'], marker='o', linestyle='-', color='b')

plt.xticks(np.arange(6, 10, 1), ['Jun', 'Jul', 'Ago', 'Set'])

plt.title('Número de ativações de sinistro nos meses de inverno')
plt.xlabel('Mês')
plt.ylabel('Número de ativações')
plt.grid(True)
plt.show()

É importante destacar que, ao filtrar os dados apenas para os meses de inverno, encontramos um total de 113 registros.

***

# Conclusão 

Com base em todos os gráficos e análises desenvolvidos, podemos concluir que, ao longo do ano, ocorrem mais de 300 consultas relacionadas a doenças respiratórias e condições semelhantes. A maior concentração dessas consultas ocorre nos meses de inverno, que correspondem a cerca de um terço do total anual. Assim, podemos confirmar que nossas hipóteses estavam corretas: a busca por tratamentos respiratórios realmente aumenta durante o inverno em comparação com os outros meses do ano.